# Create an external Spark table based on existing parquet files on the data lake

When executed, this notebook creates a shared metadata table in a Synapse lake database. This table is managed in Spark, but can also be queried using Synapse Serverless SQL. [Read more about shared metadata tables](https://learn.microsoft.com/en-us/azure/synapse-analytics/metadata/table).

The table created by this notebook is an external table, meaning that it is based on existing data on the data lake. Data is not copied or duplicated when creating the table. The notebook will create a database with the name of the data lake container, if it does not exist yet. If a table with the same name already exists, no new table will be created.

This notebook is primarily meant to be run by the Consolidation_OneEntity pipeline. If you are running the notebook manually, make sure to uncomment and fill in the parameters in the next cell.

In [ ]:
# These parameters are automatically provided by the calling pipeline. Uncomment and provide parameters if run manually!
# container_name = ''
# entity_name = ''

In [3]:
# Retrieve storage account name from Linked Service

import re
import json
linked_service_name = 'AzureDataLakeStorage'
linked_service_props = mssparkutils.credentials.getPropertiesAll(linked_service_name)
ls_props_json = json.loads(linked_service_props)
ls_endpoint = ls_props_json['Endpoint']
print(f'Found endpoint {ls_endpoint}')
re_match = re.search('https://(.*).dfs.core.windows.net', ls_endpoint)
storage_account_name = re_match.group(1)

In [ ]:
# Create database

db_name = re.sub('[^a-zA-Z0-9_]', '_', container_name) # remove invalid characters and replace them with underscores
query = f'CREATE DATABASE IF NOT EXISTS `{db_name}`'
spark.sql(query)

In [2]:
# Create table

table_name = re.sub('[^a-zA-Z0-9_]', '_', entity_name) # remove invalid characters and replace them with underscores
location = f'abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/data/{entity_name}/'
query = f'CREATE TABLE IF NOT EXISTS `{db_name}`.`{table_name}` USING Parquet LOCATION "{location}"'
spark.sql(query)